In [123]:
a = [0,0,0,0,0]


7

In [124]:
n = 7
a = [ 0 for i in range(n)] 
a


[0, 0, 0, 0, 0, 0, 0]

In [125]:
a = [ [0 for i in range(n) ] ] 
a
len(a[0])

7

In [128]:
n=5
a = [ [0 for i in range(n) ] for j in  range(n) ] 
a

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [132]:
n=5
a = [ [0]*n for j in  range(n) ] 
a

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [133]:
row=0
col=-1
count=1

for i in range(n):
   col += 1
   a[row][col] = count
   count += 1
a

[[1, 2, 3, 4, 5],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [134]:
col

4

In [135]:
row

0

In [136]:
n=5
a = [ [0]*n for j in  range(n) ] 
row=0
col=-1
count=1
num = n

for i in range(num):
   col += 1
   a[row][col] = count
   count += 1

num -= 1

for i in range(num):
   row += 1
   a[row][col] = count
   count += 1
a

[[1, 2, 3, 4, 5],
 [0, 0, 0, 0, 6],
 [0, 0, 0, 0, 7],
 [0, 0, 0, 0, 8],
 [0, 0, 0, 0, 9]]

In [139]:
def  display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()

n=5
a = [ [0]*n for j in  range(n) ] 
row=0
col=-1
count=1
num = n

for i in range(num):
   col += 1
   a[row][col] = count
   count += 1

num -= 1

for i in range(num):
   row += 1
   a[row][col] = count
   count += 1

for i in range(num):
   col -= 1
   a[row][col] = count
   count += 1

display(a)

   1   2   3   4   5
   0   0   0   0   6
   0   0   0   0   7
   0   0   0   0   8
  13  12  11  10   9


In [141]:
def  display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()

n=5
a = [ [0]*n for j in  range(n) ] 
row=0
col=-1
count=1
num = n

for i in range(num):
   col += 1
   a[row][col] = count
   count += 1

num -= 1

for i in range(num):
   row += 1
   a[row][col] = count
   count += 1

for i in range(num):
   col -= 1
   a[row][col] = count
   count += 1

num -= 1

for i in range(num):
   row -= 1
   a[row][col] = count
   count += 1

display(a)

   1   2   3   4   5   6   7   8   9  10  11
  40   0   0   0   0   0   0   0   0   0  12
  39   0   0   0   0   0   0   0   0   0  13
  38   0   0   0   0   0   0   0   0   0  14
  37   0   0   0   0   0   0   0   0   0  15
  36   0   0   0   0   0   0   0   0   0  16
  35   0   0   0   0   0   0   0   0   0  17
  34   0   0   0   0   0   0   0   0   0  18
  33   0   0   0   0   0   0   0   0   0  19
  32   0   0   0   0   0   0   0   0   0  20
  31  30  29  28  27  26  25  24  23  22  21


In [144]:
def  display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()

n=11
a = [ [0]*n for j in  range(n) ] 
row=0
col=-1
count=1
num = n

while True:
    for i in range(num):
       col += 1
       a[row][col] = count
       count += 1

    num -= 1
    if num==0 :
        break

    for i in range(num):
       row += 1
       a[row][col] = count
       count += 1

    for i in range(num):
       col -= 1
       a[row][col] = count
       count += 1

    num -= 1
    if num==0 :
        break

    for i in range(num):
       row -= 1
       a[row][col] = count
       count += 1

display(a)

   1   2   3   4   5   6   7   8   9  10  11
  40  41  42  43  44  45  46  47  48  49  12
  39  72  73  74  75  76  77  78  79  50  13
  38  71  96  97  98  99 100 101  80  51  14
  37  70  95 112 113 114 115 102  81  52  15
  36  69  94 111 120 121 116 103  82  53  16
  35  68  93 110 119 118 117 104  83  54  17
  34  67  92 109 108 107 106 105  84  55  18
  33  66  91  90  89  88  87  86  85  56  19
  32  65  64  63  62  61  60  59  58  57  20
  31  30  29  28  27  26  25  24  23  22  21


In [145]:
def  display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()

n=11
a = [ [0]*n for j in  range(n) ] 
row=0
col=-1
count=1
num = n
sign=1

while True:
    for i in range(num):
       col += sign
       a[row][col] = count
       count += 1

    num -= 1
    if num==0 :
        break

    for i in range(num):
       row += sign
       a[row][col] = count
       count += 1

    sign = -sign

display(a)

   1   2   3   4   5   6   7   8   9  10  11
  40  41  42  43  44  45  46  47  48  49  12
  39  72  73  74  75  76  77  78  79  50  13
  38  71  96  97  98  99 100 101  80  51  14
  37  70  95 112 113 114 115 102  81  52  15
  36  69  94 111 120 121 116 103  82  53  16
  35  68  93 110 119 118 117 104  83  54  17
  34  67  92 109 108 107 106 105  84  55  18
  33  66  91  90  89  88  87  86  85  56  19
  32  65  64  63  62  61  60  59  58  57  20
  31  30  29  28  27  26  25  24  23  22  21


In [148]:
def  display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        
def init(n):
    a = [ [0]*n for j in  range(n) ] 
    return a

def fill(a, n):
    count=1
    for i in range(n):
        for j in range(n):
            a[i][j] = count
            count += 1

n=5
a = init(n)
fill(a, n)
display(a)


   1   2   3   4   5
   6   7   8   9  10
  11  12  13  14  15
  16  17  18  19  20
  21  22  23  24  25


In [149]:
def  display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        
def init(n):
    a = [ [0]*n for j in  range(n) ] 
    return a

def fill(a, n):
    count=1
    for i in range(n):
        for j in range(n-i):
            a[i+j][i] = count
            count += 1

n=5
a = init(n)
fill(a, n)
display(a)

   1   0   0   0   0
   2   6   0   0   0
   3   7  10   0   0
   4   8  11  13   0
   5   9  12  14  15


In [152]:
def  display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        
def init(n):
    a = [ [0]*n for j in  range(n) ] 
    return a

def fill(a, n):
    count=1
    for i in range(n):
        for j in range(n-i):
            a[i][n-i-1-j] = count
            count += 1


n=5
a = init(n)
fill(a, n)
display(a)

   5   4   3   2   1
   9   8   7   6   0
  12  11  10   0   0
  14  13   0   0   0
  15   0   0   0   0


In [153]:
def  display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        
def init(n):
    a = [ [0]*n for j in  range(n) ] 
    return a

def fill(a, n):
    count=1
    for i in range(n):
        for j in range(i+1):
            a[i-j][j] = count
            count += 1



n=5
a = init(n)
fill(a, n)
display(a)

   1   3   6  10  15
   2   5   9  14   0
   4   8  13   0   0
   7  12   0   0   0
  11   0   0   0   0


In [156]:
def  display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        
def init(n):
    a = [ [0]*n for j in  range(n) ] 
    return a

def fill(a, n):
    count=1
    for k in range(2*n-1):
        for i in range(n):
            for j in range(n):
                if (i+j)==k : 
                    a[j][i] = count
                    count += 1

n=5
a = init(n)
fill(a, n)
display(a)

   1   3   6  10  15  21  28  36  45  55  66
   2   5   9  14  20  27  35  44  54  65  76
   4   8  13  19  26  34  43  53  64  75  85
   7  12  18  25  33  42  52  63  74  84  93
  11  17  24  32  41  51  62  73  83  92 100
  16  23  31  40  50  61  72  82  91  99 106
  22  30  39  49  60  71  81  90  98 105 111
  29  38  48  59  70  80  89  97 104 110 115
  37  47  58  69  79  88  96 103 109 114 118
  46  57  68  78  87  95 102 108 113 117 120
  56  67  77  86  94 101 107 112 116 119 121


In [157]:
def  display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        
def init(n):
    a = [ [0]*n for j in  range(n) ] 
    return a

def fill(a, n):
    count=1
    for i in range(n):
        for j in range(n-i*2):
            a[i+j][i] = count
            count += 1


n=5
a = init(n)
fill(a, n)
display(a)

   1   0   0   0   0
   2   6   0   0   0
   3   7   9   0   0
   4   8   0   0   0
   5   0   0   0   0


In [159]:
def  display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        
def init(n):
    a = [ [0]*n for j in  range(n) ] 
    return a

def fill(a, n):
    count=1
    inc=0
    flag=1
    for i in range(n):
        for j in range(n-inc*2):
            a[inc+j][i] = count
            count += 1
        if n//2 == i:
            flag = -flag
        inc += flag

n=5
a = init(n)
fill(a, n)
display(a)

   1   0   0   0   0   0   0   0   0   0  61
   2  12   0   0   0   0   0   0   0  52  62
   3  13  21   0   0   0   0   0  45  53  63
   4  14  22  28   0   0   0  40  46  54  64
   5  15  23  29  33   0  37  41  47  55  65
   6  16  24  30  34  36  38  42  48  56  66
   7  17  25  31  35   0  39  43  49  57  67
   8  18  26  32   0   0   0  44  50  58  68
   9  19  27   0   0   0   0   0  51  59  69
  10  20   0   0   0   0   0   0   0  60  70
  11   0   0   0   0   0   0   0   0   0  71


### 클래스 만들기

In [160]:
class FourCal:
    def add(self,a,b):
        result = a+b
        return result

a = FourCal()
print(a.add(3,4))

7


In [161]:
class FourCal:
    def setdata(self, first, second):
        self.first = first
        self.second = second
        
    def add(self):
        result = self.first+self.second
        return result

a = FourCal()
a.setdata(3,4)
print(a.add())

7


In [175]:
class FourCal:
    def __init__(self, first, second):
        print("FourCal.__init__()")
        self.first = first
        self.second = second
        
    def setdata(self, first, second):
        self.first = first
        self.second = second
        
    def add(self):
        result = self.first+self.second
        return result
    
    def div(self):
        print("FourCal.div()")
        result = self.first/self.second
        return result

a = FourCal(3,0)
print(a.div())

FourCal.__init__()
FourCal.div()


ZeroDivisionError: division by zero

In [176]:
class MoreFourCal(FourCal):
    def pow(self):
        result = self.first ** self.second
        return result
    
    def div(self):
        print("MoreFourCal.div()")
        if(self.second==0):
            print("분모는 0으로 지정하면 안되요...")
            return
        
        result = self.first/self.second
        return result

a = MoreFourCal(4,0)
a.div()

FourCal.__init__()
MoreFourCal.div()
분모는 0으로 지정하면 안되요...


In [181]:
def foo():
    print("foo()")
    
def foo(a):
    print("foo(a)")
    
# foo()
# foo(1)

TypeError: foo() missing 1 required positional argument: 'a'

### 클래스 변수

In [182]:
class Family:
    lastname = '김'
    def foo():
        name = '홍길동'
        
print(Family.lastname)
    

김


In [183]:
a = Family()
print(a.lastname)

김


In [184]:
b = Family()
print(b.lastname)

김


In [185]:
Family.lastname = '박'

In [186]:
print(a.lastname)

박


In [187]:
print(b.lastname)

박


In [188]:
id(Family.lastname)

2039502611600

In [189]:
id(a.lastname)

2039502611600

In [190]:
id(b.lastname)

2039502611600

### 모듈

In [200]:
import mod1
print(mod1.add(1,2))
print(mod1.sub(1,2))

mod1: add()
3
mod1: sub()
-1


In [201]:
from  mod1  import add
print(add(1,2))

mod1: add()
3


In [202]:
from  mod1  import add,sub
print(add(1,2))
print(sub(1,2))

mod1: add()
3
mod1: sub()
-1


In [203]:
from  mod1  import *
print(add(1,2))
print(sub(1,2))

mod1: add()
3
mod1: sub()
-1


In [204]:
print(__name__)

__main__


In [205]:
add(1,2)

mod1: add()


3

In [206]:
from  mod1  import *
add(1,2)

mod1: add()


3

In [209]:
from  mod1  import *

In [210]:
print(mod1.__name__)

mod1


### 예외처리

In [212]:
open("나없는파일1", "r")

FileNotFoundError: [Errno 2] No such file or directory: '나없는파일1'

In [213]:
4 / 0

ZeroDivisionError: division by zero

In [214]:
a = [1,2,3]
a[3]

IndexError: list index out of range

In [215]:
try : 
    4 / 0
except  ZeroDivisionError as e : 
    print(e)

division by zero


In [216]:
f  = open('foo.txt', 'w')
try : 
    4 / 0
finally:  
    f.close()
    print("파일 종료")

파일 종료


ZeroDivisionError: division by zero

### 여러개의 오류 처리하기 

In [224]:
try:
    4/0
    a = [1,2]
    a[3] = 4
    
except  ZeroDivisionError as e: 
    print("0으로 나눌수 없습니다.")
except IndexError:
    print("인덱싱할 수 없습니다.")

0으로 나눌수 없습니다.


In [226]:
try:
    
    a = [1,2]
    a[3] = 4
    4/0
    
except  (ZeroDivisionError,IndexError)  as e: 
    print(e)


list assignment index out of range


### 오류 회피하기 

In [229]:
try : 
    f = open("나없는파일3", "r")
except FileNotFoundError:
    pass

### 오류 일부러 발생시키기

In [231]:
class Bird:
    def fly(self):
        raise  NotImplementedError
        
b = Bird()
b.fly()

NotImplementedError: 

In [233]:
class Bird:
    def fly(self):
        raise  NotImplementedError

class Eagle(Bird):
    pass
#     def fly(self):
#         pass
    
e = Eagle()
e.fly()

NotImplementedError: 

In [235]:
class MyError(Exception):
    pass

def say_nick(nick):
    if( nick == '멍충이'):
        raise MyError()
    print(nick)
    
say_nick("호랭이")

호랭이


In [237]:
try:
    say_nick("멍충이")
except MyError:
    print('허용되지 않는 별명입니다.')

허용되지 않는 별명입니다.


In [239]:
class MyError(Exception):
    def  __str__(self):
        return "허용 되지 않는 별명입니다."

def say_nick(nick):
    if( nick == '멍충이'):
        raise MyError()
    print(nick)
    
try:
    say_nick("멍충이")
except MyError as e:
    print(e)

허용 되지 않는 별명입니다.


### 내장 함수

In [243]:
for  i, value in enumerate(['body', 'foo', 'bar']):
    print(i, value)

0 body
1 foo
2 bar


In [245]:
a = ['body', 'foo', 'bar']
b = [100,200,300,400]

for  i in range(4):
    print(a[i], b[i])

body 100
foo 200
bar 300


IndexError: list index out of range

In [246]:
a = ['body', 'foo', 'bar']
b = [100,200,300,400]

for  temp in zip(a, b):
    print(temp)

('body', 100)
('foo', 200)
('bar', 300)


In [247]:
list(zip([1,2,3],[4,5,6]))

[(1, 4), (2, 5), (3, 6)]

In [248]:
list(zip([1,2,3],[4,5,6],[7,8,9]))

[(1, 4, 7), (2, 5, 8), (3, 6, 9)]

In [249]:
list(zip("abc", "def"))

[('a', 'd'), ('b', 'e'), ('c', 'f')]

In [250]:
list(zip("abc", [10,20,30]))

[('a', 10), ('b', 20), ('c', 30)]